In [0]:
## IE-406 ML Project
## Dushyant Pathak
## Avi Gupta
## Maharshi Patel
## Pawan Baldaniya

In [40]:
## Using Google Colab for GPU usage
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive') 

Mounted at /content/drive


In [41]:
!sudo apt install openjdk-8-jdk
!sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!pip install language-check
!pip install pycontractions

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk is already the newest version (8u252-b09-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
The folder you are executing pip from can no longer be found.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent direct

In [42]:
## Importing the necessary libraries
import os
import re
import nltk
import pandas as pd
import csv
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, f1_score
from sklearn.svm import LinearSVC

!pip install Unidecode
!pip install Word2number
from bs4 import BeautifulSoup
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import unidecode
from word2number import w2n
from pycontractions import Contractions
import gensim.downloader as api
from nltk.tokenize import TweetTokenizer
import string
import re
nltk.download('stopwords')
nltk.download('wordnet')

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
The folder you are executing pip from can no longer be found.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
The folder you are executing pip from can no longer be found.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
os.chdir('/content/drive/My Drive/hate-con-proj')

In [0]:
## Train data prep
train_set = pd.read_csv('train.tsv', delimiter='\t', encoding='utf-8')
train_list = list(train_set['text'])
train = train_set[train_set['task_1'] == 'HOF']
train_frame = {0:train_list,1:list(train['text']),2:list(train['text'])}
dict_train = {0:train_set['task_1'],1:train['task_2'],2:train['task_3']}

In [0]:
## Test data prep
test_set = pd.read_csv('test.tsv', delimiter='\t', encoding='utf-8')
test_list = list(test_set['text'])
test = test_set[test_set['task_1'] == 'HOF']
test_frame = {0:test_list,1:list(test['text']),2:list(test['text'])}
dict_test = {0:test_set['task_1'],1:test['task_2'],2:test['task_3']}

We have to remove contractions('ve->have) and stopwords(like the,at, in).

In [46]:
## Pre-trained Glove Embeddings
glove_model = api.load("glove-twitter-25")
contractions = Contractions(kv_model=glove_model)
contractions.load_models()
stop_words = stopwords.words('english')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
## Preprocessing
def pre(text):
    
    ## parsing the text
    part = BeautifulSoup(text, "html.parser")
    text = part.get_text(separator=" ")
    text = text.strip()
    text = ' '.join(text.split())
    text = unidecode.unidecode(text)
    ## expanding the contractions
    text = list(contractions.expand_texts([text], precise=True))[0]
    text = re.sub('[^a-zA-Z0-9 \n\.]', '', text)
    text = re.sub(r'\d', '',text)
    text = text.lower()

    ## tokenization using TweetTokenizer
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text) #tokenise text
    
    for w in tokens:
      if w in stop_words:
        tokens.remove(w)
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(i) for i in tokens]
    text = " ".join(tokens)
    return text

In [0]:
vocab = []
for data in train_list:
    for x in data.split(" "):
        vocab.append(x)
vocab = list(set(vocab))

for i in range(3):
    for j in range(len(train_frame[i])):
        train_frame[i][j] = pre(train_frame[i][j])
    for j in range(len(test_frame[i])):
        test_frame[i][j] = pre(test_frame[i][j])

In [0]:
## Tf-IDF
count_vect = CountVectorizer(analyzer = 'word',vocabulary = vocab)
tfidf_transformer = TfidfTransformer()

In [50]:
## Naive Bayes Classifier

dict_train_2 = {}
dict_test_2 = {}
for i in range(3):
    count_train = count_vect.fit_transform(train_frame[i])
    tfidf_train = tfidf_transformer.fit_transform(count_train)
    count_test = count_vect.fit_transform(test_frame[i])
    tfidf_test = tfidf_transformer.fit_transform(count_test)
    le = LabelEncoder()
    le.fit(list(dict_train[i]))
    dict_train_2[i] = le.transform(dict_train[i].astype(str))
    cl_2 = MultinomialNB().fit(tfidf_train, dict_train_2[i])
    predictor = cl_2.predict(tfidf_test)
    dict_test_2[i] = le.transform(dict_test[i].astype(str))
    print(classification_report(dict_test_2[i], predictor))

              precision    recall  f1-score   support

           0       0.84      0.09      0.17       288
           1       0.77      0.99      0.87       865

    accuracy                           0.77      1153
   macro avg       0.81      0.54      0.52      1153
weighted avg       0.79      0.77      0.69      1153

              precision    recall  f1-score   support

           0       0.44      0.99      0.61       124
           1       0.00      0.00      0.00        71
           2       0.67      0.04      0.08        93

    accuracy                           0.44       288
   macro avg       0.37      0.34      0.23       288
weighted avg       0.40      0.44      0.29       288

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       245
           1       0.00      0.00      0.00        43

    accuracy                           0.85       288
   macro avg       0.43      0.50      0.46       288
weighted avg       0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [51]:
## SGD(Stochastic Gradient Descent) Classifier
dict_train_2 = {}
dict_test_2 = {}
for i in range(3):
    count_train = count_vect.fit_transform(train_frame[i])
    tfidf_train = tfidf_transformer.fit_transform(count_train)
    count_test = count_vect.fit_transform(test_frame[i])
    tfidf_test = tfidf_transformer.fit_transform(count_test)
    le = LabelEncoder()
    le.fit(list(dict_train[i]))
    dict_train_2[i] = le.transform(dict_train[i].astype(str))
    cl_2 = SGDClassifier().fit(tfidf_train, dict_train_2[i])
    predictor = cl_2.predict(tfidf_test)
    dict_test_2[i] = le.transform(dict_test[i].astype(str))
    print(classification_report(dict_test_2[i], predictor))

              precision    recall  f1-score   support

           0       0.43      0.56      0.49       288
           1       0.84      0.76      0.79       865

    accuracy                           0.71      1153
   macro avg       0.63      0.66      0.64      1153
weighted avg       0.74      0.71      0.72      1153

              precision    recall  f1-score   support

           0       0.52      0.60      0.55       124
           1       0.36      0.23      0.28        71
           2       0.55      0.59      0.57        93

    accuracy                           0.50       288
   macro avg       0.47      0.47      0.47       288
weighted avg       0.49      0.50      0.49       288

              precision    recall  f1-score   support

           0       0.85      0.95      0.90       245
           1       0.20      0.07      0.10        43

    accuracy                           0.82       288
   macro avg       0.53      0.51      0.50       288
weighted avg       0

In [0]:
## With Pretrained embeddings - GloVe
embed = {}
with open("pre-trained.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        v = np.asarray(values[1:], "float32")
        embed[word] = v

In [0]:
## Train data prep
train_set = pd.read_csv('train.tsv', delimiter='\t', encoding='utf-8')
train_list = list(train_set['text'])
train = train_set[train_set['task_1'] == 'HOF']
train_frame = {0:train_list,1:list(train['text']),2:list(train['text'])}
dict_train = {0:train_set['task_1'],1:train['task_2'],2:train['task_3']}
## Test data prep
test_set = pd.read_csv('test.tsv', delimiter='\t', encoding='utf-8')
test_list = list(test_set['text'])
test = test_set[test_set['task_1'] == 'HOF']
test_frame = {0:test_list,1:list(test['text']),2:list(test['text'])}
dict_test = {0:test_set['task_1'],1:test['task_2'],2:test['task_3']}


In [0]:
vocab = []
for data in train_list:
    for x in data.split(" "):
        vocab.append(x)
vocab = list(set(vocab))

for i in range(3):
    for j in range(len(train_frame[i])):
        train_frame[i][j] = pre(train_frame[i][j])
    for j in range(len(test_frame[i])):
        test_frame[i][j] = pre(test_frame[i][j])

In [0]:
train_frame_2 = train_frame
test_frame_2 = test_frame
for j in range(3):
    for i in range(len(train_frame_2[j])):
        train_frame_2[j][i] = pre(train_frame_2[j][i])
        zero = np.zeros((1,100))
        for word in train_frame_2[j][i]:
            try:
                zero += embed[word]
            except:
                pass
        train_frame_2[j][i] = zero/len(train_frame_2[j][i])
    for i in range(len(test_frame_2[j])):
        test_frame_2[j][i] = pre(test_frame_2[j][i])
        zero = np.zeros((1,100))
        for word in test_frame_2[j][i]:
            try:
                zero += embed[word]
            except:
                pass
        test_frame_2[j][i] = zero/len(test_frame_2[j][i])
for j in range(3):
  train_frame_2[j] = np.array(train_frame_2[j]).reshape(-1,100)
  test_frame_2[j] = np.array(test_frame_2[j]).reshape(-1,100) 

In [56]:
dict_train_2=dict_train
dict_test_2=dict_test
for i in range(3):
    le = LabelEncoder()
    le.fit(list(dict_train[i]))
    dict_train_2[i] = le.transform(dict_train_2[i].astype(str))
    cl_3 = SGDClassifier().fit(train_frame_2[i], dict_train_2[i])
    predictor = cl_3.predict(test_frame_2[i])
    dict_test_2[i] = le.transform(dict_test_2[i].astype(str))
    print(classification_report(dict_test_2[i], predictor))

              precision    recall  f1-score   support

           0       0.43      0.39      0.41       288
           1       0.80      0.83      0.82       865

    accuracy                           0.72      1153
   macro avg       0.62      0.61      0.61      1153
weighted avg       0.71      0.72      0.71      1153

              precision    recall  f1-score   support

           0       0.49      0.93      0.64       124
           1       0.60      0.04      0.08        71
           2       0.65      0.34      0.45        93

    accuracy                           0.52       288
   macro avg       0.58      0.44      0.39       288
weighted avg       0.57      0.52      0.44       288

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       245
           1       0.00      0.00      0.00        43

    accuracy                           0.85       288
   macro avg       0.43      0.50      0.46       288
weighted avg       0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-06-04 16:35:35--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.65.54
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.65.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  90.4MB/s    in 19s     

2020-06-04 16:35:55 (81.0 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [58]:
!pip install gensim
from gensim.models import KeyedVectors

In [0]:
## Train data prep
train_set = pd.read_csv('train.tsv', delimiter='\t', encoding='utf-8')
train_list = list(train_set['text'])
train = train_set[train_set['task_1'] == 'HOF']
train_frame = {0:train_list,1:list(train['text']),2:list(train['text'])}
dict_train = {0:train_set['task_1'],1:train['task_2'],2:train['task_3']}
## Test data prep
test_set = pd.read_csv('test.tsv', delimiter='\t', encoding='utf-8')
test_list = list(test_set['text'])
test = test_set[test_set['task_1'] == 'HOF']
test_frame = {0:test_list,1:list(test['text']),2:list(test['text'])}
dict_test = {0:test_set['task_1'],1:test['task_2'],2:test['task_3']}

vocab = []
for data in train_list:
    for x in data.split(" "):
        vocab.append(x)
vocab = list(set(vocab))

for i in range(3):
    for j in range(len(train_frame[i])):
        train_frame[i][j] = pre(train_frame[i][j])
    for j in range(len(test_frame[i])):
        test_frame[i][j] = pre(test_frame[i][j])


In [0]:
## With Pretrained embeddings - Word2Vec
embed = {}
with open("model.txt", 'r', encoding = 'utf-8', errors='ignore') as f:
    for line in f:
        values = line.split()
        word = values[0]
        v = np.asarray(values[1:], "float32")
        embed[word] = v

In [0]:
train_frame_3 = train_frame
test_frame_3 = test_frame
for j in range(3):
    for i in range(len(train_frame_3[j])):
        train_frame_3[j][i] = pre(train_frame_3[j][i])
        zero = np.zeros((1,100))
        for word in train_frame_3[j][i]:
            try:
                zero += embed[word]
            except:
                pass
        train_frame_3[j][i] = zero/len(train_frame_3[j][i])
    for i in range(len(test_frame_3[j])):
        test_frame_3[j][i] = pre(test_frame_3[j][i])
        zero = np.zeros((1,100))
        for word in test_frame_3[j][i]:
            try:
                zero += embed[word]
            except:
                pass
        test_frame_3[j][i] = zero/len(test_frame_3[j][i])
for j in range(3):
  train_frame_3[j] = np.array(train_frame_3[j]).reshape(-1,100)
  test_frame_3[j] = np.array(test_frame_3[j]).reshape(-1,100) 

In [62]:
dict_train_3=dict_train
dict_test_3=dict_test
for i in range(3):
    le = LabelEncoder()
    le.fit(list(dict_train[i]))
    dict_train_3[i] = le.transform(dict_train_3[i].astype(str))
    cl_4 = SGDClassifier().fit(train_frame_3[i], dict_train_3[i])
    predictor = cl_4.predict(test_frame_3[i])
    dict_test_3[i] = le.transform(dict_test_3[i].astype(str))
    print(classification_report(dict_test_3[i], predictor))

              precision    recall  f1-score   support

           0       0.44      0.01      0.03       288
           1       0.75      0.99      0.86       865

    accuracy                           0.75      1153
   macro avg       0.60      0.50      0.44      1153
weighted avg       0.67      0.75      0.65      1153

              precision    recall  f1-score   support

           0       0.53      0.62      0.57       124
           1       0.00      0.00      0.00        71
           2       0.45      0.68      0.54        93

    accuracy                           0.49       288
   macro avg       0.32      0.43      0.37       288
weighted avg       0.37      0.49      0.42       288

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       245
           1       0.00      0.00      0.00        43

    accuracy                           0.85       288
   macro avg       0.43      0.50      0.46       288
weighted avg       0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
